<a href="https://colab.research.google.com/github/Vakhranev/Compling/blob/master/Gospodi_za_chto.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd

In [4]:
f = pd.read_csv('jigsaw-toxic-comment-train.csv.zip')
f.head(20)

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0
5,00025465d4725e87,"""\n\nCongratulations from me as well, use the ...",0,0,0,0,0,0
6,0002bcb3da6cb337,COCKSUCKER BEFORE YOU PISS AROUND ON MY WORK,1,1,1,0,1,0
7,00031b1e95af7921,Your vandalism to the Matt Shirvington article...,0,0,0,0,0,0
8,00037261f536c51d,Sorry if the word 'nonsense' was offensive to ...,0,0,0,0,0,0
9,00040093b2687caa,alignment on this subject and which are contra...,0,0,0,0,0,0


Сравним ряд параметров для трёх групп:
1. Toxic: toxic=1 и toxic=0
2. Severe toxic: severe_toxic=1 и severe_toxic=0
3. Obscene: obscene=1 и obscene=0

Сравним по среднему количеству символов, среднему количеству заглавных букв (мы видим в примерах выше токсичный комментарий, написанный капсом), среднему количеству знаков пунктуации, среднему количеству цифр, а также по количеству слов (точнее — токенов) в столбце comment_text.

In [5]:
from string import punctuation
f['Количество символов'] = f['comment_text'].apply(len)
f['Количество заглавных букв'] = f['comment_text'].apply(lambda comment: len([i for i in comment if i.isupper()]))
f['Количество знаков препинания'] = f['comment_text'].apply(lambda comment: len([i for i in comment if i in punctuation]))
f['Количество цифр'] = f['comment_text'].apply(lambda comment: len([i for i in comment if i.isdigit()]))
f['Количество слов'] = f['comment_text'].apply(lambda comment: len(comment.split()))

In [6]:
f.head(20)

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate,Количество символов,Количество заглавных букв,Количество знаков препинания,Количество цифр,Количество слов
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0,264,17,10,9,43
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0,112,8,12,10,17
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0,233,4,6,0,42
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0,622,11,21,0,113
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0,67,2,5,0,13
5,00025465d4725e87,"""\n\nCongratulations from me as well, use the ...",0,0,0,0,0,0,65,1,4,0,13
6,0002bcb3da6cb337,COCKSUCKER BEFORE YOU PISS AROUND ON MY WORK,1,1,1,0,1,0,44,37,0,0,8
7,00031b1e95af7921,Your vandalism to the Matt Shirvington article...,0,0,0,0,0,0,115,4,4,0,20
8,00037261f536c51d,Sorry if the word 'nonsense' was offensive to ...,0,0,0,0,0,0,472,7,19,11,83
9,00040093b2687caa,alignment on this subject and which are contra...,0,0,0,0,0,0,70,2,0,0,12


In [30]:
def analyze(column):
  return(f.groupby(column)['Количество символов', 'Количество заглавных букв', 'Количество знаков препинания', 'Количество цифр', 'Количество слов'].agg('mean'))

In [31]:
analyze('toxic')

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  


,Количество символов,Количество заглавных букв,Количество знаков препинания,Количество цифр,Количество слов
toxic,,,,,
0,402.691178,14.484189,16.974704,3.248698,68.415161
1,280.604097,44.689675,14.257202,1.995885,48.573466


Токсичные выражения короче по символам, словам, в них меньше знаков препинания и цифр, но больше заглавных букв (за счёт комментариев, написанных капсом, видимо).

In [32]:
analyze('severe_toxic')

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  


,Количество символов,Количество заглавных букв,Количество знаков препинания,Количество цифр,Количество слов
severe_toxic,,,,,
0,389.999386,15.988131,16.591014,3.136258,66.358270
1,505.456677,173.841998,30.690112,2.293068,84.462793


У особо токсичных выражений статистика другая: в них больше символов, слов, знаков перпинания, а количество заглавных букв больше, чем в 10 раз. Только цифр меньше.

In [33]:
analyze('obscene')

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  


,Количество символов,Количество заглавных букв,Количество знаков препинания,Количество цифр,Количество слов
obscene,,,,,
0,397.332876,15.299590,16.850087,3.193029,67.559707
1,280.951730,53.490033,14.358072,2.011367,48.362109


Статистика непристойностей посвоим показателям очень близка к статистике токсичных комментариев.